In [1]:
import os
os.environ['http_proxy'] = 'http://127.0.0.1:7890'
os.environ['https_proxy'] = 'http://127.0.0.1:7890'

In [2]:
from datasets import load_dataset
from trl import SFTConfig, SFTTrainer

[2024-07-28 11:06:55,085] [INFO] [real_accelerator.py:161:get_accelerator] Setting ds_accelerator to cuda (auto detect)


### wandb

```
os.environ['WANDB_DISABLED'] = 'true'
```

- 关闭 wandb 服务


### SFTConfig

- `class SFTConfig(TrainingArguments):`
    - 继承了 TrainingArguments 类，
        - `num_train_epochs`: default 3
        - `per_device_train_batch_size`: default 8
        - `per_device_eval_batch_size`: default 8
        - `gradient_accumulation_steps`: default 1
        - `dataloader_drop_last`: default false
        - `report_to`:
            - none
            - tensorboard
            - wandb
- `dataset_text_field`: 跟 dataset 的成员对齐
- `max_seq_length`
- `output_dir='/tmp'`
- **packing=True**,
    - example packing, where multiple short examples are packed in the same input sequence to increase training efficiency. 
    - `# allows multiple shorter sequences to be packed into a single training example, maximizing the use of the model's context window.`

In [3]:
imdb = load_dataset('imdb', split='train')
imdb

Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

Dataset({
    features: ['text', 'label'],
    num_rows: 25000
})

```
sft_config = SFTConfig(
    dataset_text_field="text",
    max_seq_length=512,
    output_dir="/tmp",
)
```

### training steps

```
# 20022
dataset = load_dataset("lucasmccabe-lmi/CodeAlpaca-20k", split="train")

args = SFTConfig(output_dir="/tmp", 
                 max_seq_length=512, 
                 num_train_epochs=2, 
                 per_device_train_batch_size=4, 
                 gradient_accumulation_steps=4,
                 gradient_checkpointing=True,
                 )

trainer = SFTTrainer(
    model,
    train_dataset=dataset,
    args=args,
    formatting_func=formatting_prompts_func,
    data_collator=collator,
)
```

- training_epochs * len(dataset) / (_train_batch_size * args.gradient_accumulation_steps * args.world_size)
    - `20022 * 2 / (4*4*2)`

In [5]:
20022 * 2 / (4*4*2)

1251.375